In [1]:
import os
from langchain_groq import ChatGroq

In [2]:
GROQ_LLM = ChatGroq(api_key = os.getenv("GROQ_API_KEY"), model = "llama3-70b-8192")

In [ ]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)


In [3]:
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime
from random import randint
from langchain.tools import tool
import json
import requests
from langchain.agents import load_tools

search_tool = DuckDuckGoSearchRun()

In [5]:
search_tool.run('jesus')

'Jesus (born c. 6-4 bce, Bethlehem—died c. 30 ce, Jerusalem) was a religious leader revered in Christianity, one of the world\'s major religions.He is regarded by most Christians as the Incarnation of God. The history of Christian reflection on the teachings and nature of Jesus is examined in the article Christology.. Name and title Explore the identity of Jesus Christ based on the Bible, from His claims, prophecies, miracles, and resurrection. Learn how to know Jesus Christ as your Savior and Lord and the way to God. Check out this three hour marathon following the life of Jesus, from birth to betrayal. #JesusHisLifeSubscribe for more from The HISTORY Channel:http://histv... Jesus Is Peace. "For to us a child is born, to us a son is given, and the government will be on his shoulders. And he will be called Wonderful Counselor, Mighty God, Everlasting Father, Prince of Peace" ( Isaiah 9:6 ). For many of us, Isaiah 9:6 stirs memories of Christmas, and this is fitting for the prophecy pro

In [ ]:
class EmailAgents():
    def make_categorizing_agent(self):
        return Agent(
            role='Email Categorizing Agent',
            goal = '''take in a email from a human that has emailed out company email address and categorize it \
            into one of the following categories: \
            price_equiry - used when someone is asking for information about pricing \
            customer_complaint - used when someone is complaining about something \
            product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
            customer_feedback - used when someone is giving feedback about a product \
            off_topic when it doesnt relate to any other category''',
            backstory = '''You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way''',
            llm = GROQ_LLM,
            verbose = True,
            allow_delegation = False,
            max_iter = 5,
            memory = True,
            step_callback = lambda x: print_agent_output(x, 'Email Categorizing Agent')
        )
    
    def make_researcher_agent(self):
        return Agent(
            role = "Information Researcher Agent",
            goal = """take in a email from a human that has emailed out company email address and the category \
            that the categorizer agent gave it and decide what information you need to search for for the email writer to reply to \
            the email in a thoughtful and helpful way.
            If you DONT think a search will help just reply 'NO SEARCH NEEDED'
            If you dont find any useful info just reply 'NO USEFUL RESESARCH FOUND'
            otherwise reply with the info you found that is useful for the email writer""",
            backstory = ''' '''
            llm = GROQ_LLM,
            verbose = True,
            allow_delegation = False,
            max_iter = 5,
            memory = True,
            step_callback = lambda x: print_agent_output(x, 'Information Researcher Agent')

        )
    
    def make_email_writer_agent(self):
        return Agent(
            role = "Email Writer Agent",
            goal = '''take in a email from a human that has emailed out company email address, the category \
            that the categorizer agent gave it and the research from the research agent and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information. that hasn't been provided by the researcher or in the email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager. ''',
            backstory = '''You are a master at synthesizing a variety of information and writing a helpful email \
            that will address the customer's issues and provide them with helpful information''',
            llm = GROQ_LLM,
            verbose = True,
            max_iter = 5,
            memory = True,
            allow_delegation = False,
            step_callback = lambda x: print_agent_output(x, 'Email Writer Agent')
        )
        

